___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Mariana Abrantes

#### Gabriel Couto

#### Gabriel Miras
___

# Previsão de preços de smartphones ao dólar

___

## A. INTRODUÇÃO

Nestes últimos anos, com a instabilidade política e socioeconômica do Brasil, o preço do dólar está em constante variação. Consequentemente, os valores sobre produtos estrangeiros também mudam. Ao visualizar esse cenário, gostaríamos de analisar essa influência da alteração do valor do dólar sobre produtos da área tecnológica, em nosso caso smartphones. 

___

## Criando a base de dados


Para a reaização deste trabalho, foi preciso duas bases de dados. Separamos a cotação do dólar durante o ano inteiro de 2018 até maio de 2019, e escolhemos 8 modelos de smartphones de 2 marcas diferentes lançados recentemente. Iremos compará-los e verificar suas caracteristicas e como elas são decisivas para o preço do smartphone.

In [2]:
import pandas as pd
import requests
import os
import re
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

#------------------------------------------- Preço do dolar --------------------------------------------------------------------
dolar19 = pd.read_csv('dolar 19.csv')
dolar18 = pd.read_csv('dolar 18.csv')
dolar   = dolar19.append(dolar18)
dolar['Data'] = dolar['Data'].str.replace('.','/')
dolar.index = dolar['Data']
dol = pd.DataFrame(columns = dolar.index)
dol = dol.append(dolar.T.loc['Último',:])

In [3]:
#Recebe uma URL e retorna o código HTML da página (Só é chamada pela função baixarBD)
def baixarhtml(url):
    html = requests.get(url).text.splitlines()
    return html

#Recebe url do Buscape e retorna um Dataframe 
def baixarBD(url):
    #Pegar ID do produto do buscape usando a url fornecida
    code = baixarhtml(url)
    for i in range(len(code)):
        igual = re.findall('"productId":[0-9]*', code[i])
        if len(igual)>0:
            id = igual[0].split(':')[1]
            
    #Baixar BD dos preços
    site = "https://www.buscape.com.br/async/history/idu/" + id + "/days/365"
    code = baixarhtml(site)
    
    #Converte código e cria tabela
    c = code[0].split('"historicos":')
    json_file = c[1][:-1]
    bd = pd.read_json(json_file)
    return bd.loc[:,['data','precomed','precomin']]

    
#Recebe lista com o nome do produto e links e baixa todos eles
def df_min(lista):
    for num in range(len(lista)):
        produto, link = lista[num][0], lista[num][1]
        df0 = baixarBD(link)
        df = df0.loc[:,['precomin']].T #Cria Tabela de precomin por data
        df.index = [produto]
        if num == 0:
            bd = pd.DataFrame(columns = range(len(df)))
            bd = bd.append(df)
        else:
            bd = bd.append(df)
            
    bd.columns = list(df0['data'])
    return bd

In [14]:
leitura_samsung = pd.ExcelFile('caracteristica_samsung.xlsx')
caracteristicas_samsumg = pd.read_excel(leitura_samsung)

leitura_apple = pd.ExcelFile('caracteristicas_iphone.xlsx')
caracteristicas_apple = pd.read_excel(leitura_apple)

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [4]:
dol

Data,09/05/2019,08/05/2019,07/05/2019,06/05/2019,05/05/2019,03/05/2019,02/05/2019,01/05/2019,30/04/2019,29/04/2019,...,12/01/2018,11/01/2018,10/01/2018,09/01/2018,08/01/2018,05/01/2018,04/01/2018,03/01/2018,02/01/2018,01/01/2018
Último,"3,9420","3,9285","3,9710","3,9695","3,9383","3,9382","3,9661","3,9202","3,9215","3,9456",...,"3,2064","3,2155","3,2283","3,2483","3,2407","3,2292","3,2346","3,2370","3,2591","3,3125"


In [5]:
links = [
    ['Iphone X',          "https://www.buscape.com.br/smartphone-apple-iphone-x-64gb"],
    ['Iphone 8 Plus',     "https://www.buscape.com.br/smartphone-apple-iphone-8-plus-64gb"],
    ['Iphone 7',          "https://www.buscape.com.br/smartphone-apple-iphone-7-32gb"],
    ['Iphone 7 Plus',     "https://www.buscape.com.br/smartphone-apple-iphone-7-plus-128gb"],
    ['Galaxy S8',         "https://www.buscape.com.br/smartphone-samsung-galaxy-s8-sm-g950-64gb"],
    ['Galaxy S8 Plus',    "https://www.buscape.com.br/smartphone-samsung-galaxy-s8-plus-sm-g955-128gb"],
    ['Galaxy S9',         "https://www.buscape.com.br/smartphone-samsung-galaxy-s9-sm-g960-128gb"],
    ['Galaxy S9 Plus',    "https://www.buscape.com.br/smartphone-samsung-galaxy-s9-plus-sm-g965-128gb"]
]

In [6]:
dfProdutos = df_min(links)

In [7]:
dfProdutos.T.head(5)

,Iphone X,Iphone 8 Plus,Iphone 7,Iphone 7 Plus,Galaxy S8,Galaxy S8 Plus,Galaxy S9,Galaxy S9 Plus
22/05/2018,5099.0,3554.10,2449.79,3132.9,2519.99,3533.07,3449.0,4097.00
23/05/2018,5099.0,3475.12,2375.10,3089.0,2519.99,3533.07,3449.0,3999.00
24/05/2018,5099.0,3484.15,2375.10,3089.0,2429.19,3533.07,3449.0,4049.00
25/05/2018,5219.1,3484.15,2375.10,3059.0,2519.99,2899.00,3449.0,4084.05
26/05/2018,5099.0,3519.11,2375.10,3059.0,2479.90,3533.07,3449.0,4049.00


### BASE DE DADOS

### ANÁLISE DESCRITIVA

Em nossa base de dados buscamos os preços de telefones das marcas Apple e Samsung, sendo eles Iphone X, Iphone 8 Plus, Iphone 7 plus, iphone 7, Galaxy S8, Galaxy S8 Plus, Galaxy S9 e Galaxy S9 Plus. Além disso, pegamos também as características dos modelos escolhidos para que nossa análise seja mais específica, pois ao mudar o tamanho da memória do celular, por exemplo, o preço se altera também. Desse modo, conseguiremos analisar qual a interferência da alteração do dólar nos valores dos smartphones. 

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS

https://br.investing.com/ $\rightarrow$ site para busca da cotação dos dolores

https://www.buscape.com.br $\rightarrow$ site para busca de histórico dos preços